In [3]:
import pandas as pd
import numpy as np
import stanza
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2

In [4]:
df_train = pd.read_csv('./datasets/train_content.csv')
df_train.head()

,_id,date,tags,title,hubs_title,description,name,hub,png,nick,url,content,favs_lognorm
0,https://geektimes.ru/post/21866/,2008-03-17T18:55:00.000Z,"['eeepc', 'asus', 'ЭТО', 'эльдорадо', 'ура']",eeePC в продаже. Да. Правда.,Железо,"Итак, если 3 дня назад я отписался то что в пр...",Сергей 'pokatusher',hub/hardware,https://habrastorage.org/getpro/habr/olpicture...,@M_org,https://geektimes.ru/users/M_org,"Итак, если 3 дня назад я <a href=""http://habra...",2.484907
1,https://habrahabr.ru/company/aladdinrd/blog/30...,2016-06-24T13:02:00.000Z,"['Интеграция', 'шифрование', 'Windows', 'Win32...",«Разрубить Гордиев узел» или преодоление пробл...,Системное программирование,Современная операционная система это сложный и...,Аладдин Р.Д.,hub/system_programming,https://habrastorage.org/files/cbd/cf9/5ff/cbd...,NaN,https://habrahabr.ru/company/aladdinrd,Современная операционная система это сложный и...,4.174387
2,https://geektimes.ru/post/92887/,2010-05-06T10:00:00.000Z,"['mc', 'midnight commander', 'diffview', 'merg...",Релиз Midnight Commander 4.7.2 и 4.7.0.5,Чёрная дыра,Спустя 2 месяца упорных трудов вышла новая вер...,Илья Маслаков,hub/closet,https://geektimes.ru/images/logo.png,@smind,https://geektimes.ru/users/smind,Спустя 2 месяца упорных трудов вышла новая вер...,0.000000
3,https://habrahabr.ru/post/290824/,2015-05-22T11:01:00.000Z,"['бизнес-модель', 'бизнес-моделирование']",7 шагов для постройки правильной бизнес-модели,Интернет-маркетинг,Большинство IT предпринимателей сосредотачиваю...,Александр,hub/internetmarketing,https://habrastorage.org/files/50e/211/9a0/50e...,@jasiejames,https://habrahabr.ru/users/jasiejames,"<img src=""https://habrastorage.org/files/50e/2...",3.496508
4,https://habrahabr.ru/post/190088/,2014-09-04T00:32:00.000Z,"['python', 'flask', 'mongodb', 'pet-project']",Thunderargs: практика использования. Часть 2,Программирование,История создания Часть 1 Добрый день. Вкратце...,Данияр Супиев,hub/programming,https://habrahabr.ru/i/habralogo.jpg,@uthunderbird,https://habrahabr.ru/users/uthunderbird,"<a href=""http://habrahabr.ru/post/223041/"">Ист...",3.688879


In [14]:
def deleted_symbol(text):
    pattern_end_html  = r'</\w*>|<\w*>' 
    pattern_start_html = r'<.*>|\n|\r|\r'
    pattern  =r"[^а-яА-Яa-zA-Z]"
    text = re.sub(pattern_end_html,' ' ,text)
    text = re.sub(pattern_start_html,'' ,text)
 
    text = re.sub("[^а-яА-Яa-zA-Z]"," ", text)
 
    return text

def tokenize(text):
    text = deleted_symbol(text)
    morph = pymorphy2.MorphAnalyzer()
    stop_words = set(stopwords.words('english')) | set(stopwords.words('russian'))
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if not token in stop_words]
#     tokens = [morph.parse(str(token))[0].normal_form for token in  tokens]
    return tokens


In [6]:
df_train['date'] = df_train['date'].apply(pd.to_datetime)
df_train['year'] = df_train['date'].apply(lambda x:x.year)
df_train['month'] = df_train['date'].apply(lambda x:x.month)

In [16]:
df_train['content'] = df_train['content'].fillna('')
data  = df_train['content'][:1000].apply(tokenize)


In [18]:
y = df_train['favs_lognorm'][:1000]

In [19]:
from gensim.models import word2vec

model = word2vec.Word2Vec(data,size = 300,window=10,workers = 4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/home/hvidsmen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [20]:
model.wv.most_similar(positive=['open', 'data','science','best'])

[('iptables', 0.9999215006828308),
 ('invoke', 0.9999042749404907),
 ('reader', 0.9999027848243713),
 ('controller', 0.9998915195465088),
 ('tools', 0.9998818039894104),
 ('ppp', 0.9998477697372437),
 ('reject', 0.9998301267623901),
 ('ds', 0.9998178482055664),
 ('restrictions', 0.9998090863227844),
 ('css', 0.9997862577438354)]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

class TfIdfVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])
    
class MeanVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [23]:
from sklearn.model_selection import train_test_split

In [25]:
content_mean = MeanVectorizer(w2v).transform(data) 
X = content_mean

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)


In [26]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

model = Ridge(alpha=1, random_state=7)
model.fit(X_train,y_train)
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)
print('Ошибка на трейне',mean_squared_error(y_train, train_preds))
print('Ошибка на валидации',mean_squared_error(y_test, test_preds))


Ошибка на трейне 2.666873687382702
Ошибка на валидации 2.669514530762718


In [28]:
data_mean_tfidf = TfIdfVectorizer(w2v).fit(data).transform(data)
X = data_mean_tfidf
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [29]:
model = Ridge(alpha=1, random_state=7)
model.fit(X_train,y_train)
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)
print('Ошибка на трейне',mean_squared_error(y_train, train_preds))
print('Ошибка на валидации',mean_squared_error(y_test, test_preds))

Ошибка на трейне 2.475616623074247
Ошибка на валидации 2.5053764731548522


In [30]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


Часть с bag of word и реализацией в sklearn

In [59]:
tfidf = TfidfVectorizer(analyzer=lambda x: x)
tfidf.fit(data)
X = tfidf.transform(data)
tf_idf_bow = X.toarray()


In [64]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer=lambda x: x)
bow = cv.fit_transform(data)
